In [1]:
# !pip install langdetect googletrans==4.0.0-rc1

In [2]:
import pandas as pd
from langdetect import detect
from googletrans import Translator, LANGUAGES

In [25]:
df = pd.read_csv("../Data_Sets/electronics_reviews.csv")

In [4]:
df.head(2)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Buen vendedor,Todo perfecto,[],B06XWZWYVP,B071R715MZ,AGTUTBAETJADNOTZNQMDM6MD7EUA,1550165327367,0,True
1,4.0,Easy to set up,Sound is good quality. I like it that although...,"[{'attachment_type': 'IMAGE', 'large_image_url...",B094HGQKFY,B096N43C98,AFM6YFPTEMBQ5EJYNX5QIFV6ZTPQ,1644514482866,2,True


# Using Google translator

In [5]:
# Initialize the translator
translator = Translator()

def detect_and_translate(text):
    try:
        # Detect language
        lang = detect(text)
        if lang != 'en':
            # Translate to English
            translated = translator.translate(text, src=lang, dest='en').text
            return translated
        return text
    except Exception as e:
        return str(e)

# Apply the detect_and_translate function to the text column
print(df['title'][:5].apply(detect_and_translate))

0                 Buen Vendedor
1                Easy to set up
2                    Five Stars
3           Clear sharp picture
4    Great Value, Great Product
Name: title, dtype: object


In [6]:
print(df['text'][:5].apply(detect_and_translate))

0                                          All perfect
1    Sound is good quality. I like it that although...
2                DOES EXACTLY WHAT IT IS INTENDED FOR.
3    The TV antennae is quick to install and driver...
4    Super easy to setup, took less then a minute. ...
Name: text, dtype: object


# Using open AI

In [7]:
# !pip install --upgrade httpx openai

In [8]:
import openai
import pandas as pd

# Set your OpenAI API key
openai.api_key = 'your-key' # ???

# Sample dataset
data = {
    'text': [
        'Hello, how are you?',
        'Bonjour, comment ça va?',
        'Hola, ¿cómo estás?',
        'Hallo, wie geht es dir?',
        'Ciao, come stai?',
        'Buen vendedor'
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

def detect_and_translate(text):
    try:
        # Use OpenAI API to determine the language and translate
        response = openai.Completion.create(
            model="gpt-3.5-turbo",
            prompt=f"Detect the language and translate this text to English: '{text}'",
            max_tokens=100
        )
        translated_text = response.choices[0].text.strip()
        return translated_text
    except Exception as e:
        print(f"Error processing text: {text}. Error: {e}")
        return text

# Apply the detect_and_translate function to the text column
df['text'][:5].apply(detect_and_translate)

Error processing text: Hello, how are you?. Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error processing text: Bonjour, comment ça va?. Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error processing text: Hola, ¿cómo estás?. Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error processing text: Hallo, wie geht es dir?. Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error processing text

0        Hello, how are you?
1    Bonjour, comment ça va?
2         Hola, ¿cómo estás?
3    Hallo, wie geht es dir?
4           Ciao, come stai?
Name: text, dtype: object

# Named Entity recognition using matcher

In [9]:
import pandas as pd
import spacy
from spacy.matcher import Matcher

In [10]:
# !python -m spacy download en_core_web_sm

In [11]:
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

In [12]:
# Sample dataset
# data = {
#     'review': [
#         'The phone has a great battery life, excellent camera, and loud sound.',
#         'I love the bluetooth connectivity of this speaker.',
#         'This laptop has an amazing display and the keyboard is very comfortable.',
#         'The smartwatch has good battery but the GPS is not accurate.',
#         'The headphones have great sound quality and the battery lasts long.'
#     ]
# }

# # Create a DataFrame
# df = pd.DataFrame(data)

# Define product features to look for
product_features = [
    "battery", "camera", "sound", "bluetooth", "display", "keyboard", "GPS", "headphones"
]

# Create a spaCy Matcher instance
matcher = Matcher(nlp.vocab)

# Add patterns for each product feature
for feature in product_features:
    pattern = [{"LOWER": feature.lower()}]
    matcher.add(feature, [pattern])

def extract_features(review):
    doc = nlp(review) 
    matches = matcher(doc)
    extracted_features = [doc[start:end].text for match_id, start, end in matches]
    return extracted_features

# Apply the extract_features function to the review column
df['text'][:50].apply(extract_features)

0    []
1    []
2    []
3    []
4    []
5    []
Name: text, dtype: object

In [13]:
# nltk.download('punkt')

# Using nltk POS

In [14]:
import nltk

lines = df["text"][5]
print(lines)
# function to test if something is a noun
is_noun = lambda pos: pos[:2] == 'NN'
# do the nlp stuff
tokenized = nltk.word_tokenize(lines)
nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 

print(nouns)

Buen vendedor
['Buen', 'vendedor']


# Using Yake

In [15]:
# !pip install yake

In [16]:
import yake
kw_extractor = yake.KeywordExtractor()
text = """I have spent several years carrying an Inreach and it is hard to believe how unreliable this device is. Updating firmware is a nightmare. Syncing, nightmare. User Interface is clunky. Getting a computer to both use the sync/update software and actually see the device plugged in via USB is a nightmare. You can either go with a less robust satellite beacon or go full satellite phone - either is a more reliable alternative."""
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
numOfKeywords = 10
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
    print(kw)

('carrying an Inreach', 0.012849913978126995)
('spent several years', 0.031243910890458804)
('years carrying', 0.031243910890458804)
('Inreach', 0.07288751237548687)
('nightmare', 0.07934818494128172)
('unreliable this device', 0.1705844289713273)
('spent', 0.17406120936370134)
('years', 0.17406120936370134)
('carrying', 0.17406120936370134)
('hard', 0.17406120936370134)


# Using KeyBert

In [17]:
# !pip install keybert

In [27]:
# from keybert import KeyBERT

# Using Spacy NER

In [19]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [20]:
raw_text="I LOVE my Amazon 4K Fire Stick!!!  My TV is only 5 years old but already out of date!  I couldn‚Äôt load any new apps but now I do it all on my Fire Stick, and I have only had a brief pause in streaming once, most likely due to my wifi connection.  I like it so much I have purchased a second one for one of my other TVs and I have recommended it to so many friends who were having the same issues I was with their relatively new, yet ‚Äúold‚Äù TV.  Since I already have an echo dot in my room, I just tell Alexa what I want on the TV and she does it!  Definitely one of my best purchases for 2020!!!!"

text1= NER(raw_text)

# Now, we print the data on the NEs found in this text sample.

for word in text1.ents:
    print(word.text,word.label_)
displacy.render(text1,style="ent",jupyter=True)    

LOVE ORG
Amazon 4K Fire Stick ORG
only 5 years old DATE
Fire Stick FAC
second ORDINAL
one CARDINAL
one CARDINAL
‚ PERSON
Alexa ORG
2020 DATE


In [21]:
raw_text="I have spent several years carrying an Inreach and it is hard to believe how unreliable this device is. Updating firmware is a nightmare. Syncing, nightmare. User Interface is clunky. Getting a computer to both use the sync/update software and actually see the device plugged in via USB is a nightmare. You can either go with a less robust satellite beacon or go full satellite phone - either is a more reliable alternative."


text1= NER(raw_text)

# Now, we print the data on the NEs found in this text sample.

for word in text1.ents:
    print(word.text,word.label_)
displacy.render(text1,style="ent",jupyter=True)    

several years DATE
Inreach PERSON
Interface ORG
USB ORG


In [22]:
raw_text="Sound is good quality. I like it that although I only bought one set of headphones the charger has an option for a second set incase I want to buy another one."


text1= NER(raw_text)

# Now, we print the data on the NEs found in this text sample.

for word in text1.ents:
    print(word.text,word.label_)
displacy.render(text1,style="ent",jupyter=True)    

second ORDINAL


# Combining nltk POS + Spacy NER

In [26]:
import nltk

for i in range(50):
    lines = df["title"][i] + df["text"][i]
    # print(lines)
    # function to test if something is a noun
    is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    tokenized = nltk.word_tokenize(lines)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 

    raw_text = " ".join(nouns)
#     print(raw_text)

    text1= NER(raw_text)

    for word in text1.ents:
        print(word.text,word.label_)
    print("#"*10)
    displacy.render(text1,style="ent",jupyter=True)    
    print("\n")

##########


/Users/quicktech/anaconda3/lib/python3.11/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)




upSound NORP
##########




##########




##########


...



Digital ORG
##########




Time ORG
LG ORG
years DATE
##########




investmentBest bang buck coverage PERSON
##########




##########




##########
